# 4.3.1 EDNETが提供しているEDINET APIから有価証券報告書のデータを取得
※文章はREADMEに記載した論文から抜粋しています

金融庁のEDINET（Electronic Disclosure for Investors’ NETwork）が提供している EDINET API を利用し、本稿の有価証券報告書を取得した。
 
EDINET APIとは、2019年3月に公開された有価価証券報告書データを取得するためのAPI（Application Programming Interface）のことである。
 
これにより、プログラムを用いた機械的なデータ取得をネットワークに過度な負荷をかけることなく実行可能にしている。EDINETAPIを用いたデータ取得までの手順は以下の通りに行った。
 
1.     コロナ前、コロナ過渡期、コロナ後に該当するそれぞれの日付を設定した URL にリクエストし、その日に公開されたJSON式の文書一覧のデータをレスポンスとして受け取る。
2.     レスポンスとして受け取った文書一覧から取得したい有価証券報告書のIDを読み込む。
3.    有価証券報告書のIDを設定したURLをリクエストし、 Zipファイルとして送られてきた有価証券書のデータをレスポンスとして受け取る。

In [81]:
import datetime 
import pandas as pd
import requests as req
from datetime import datetime as dt
import time
import os
import requests as req

def call_edinet_api(data_frame, data_frame_name , start_date, end_date):
    try:
        edinet_numbers_dir = call_edinet_numbers_dir(data_frame_name)
        
        if not os.path.isfile(f"{edinet_numbers_dir}/サービス業_edinet_numbers"):
            day_list = make_day_list(start_date, end_date) 
            edicode_values  = make_edicode_values(data_frame)
            industry_securities_report_doc_list_hash = make_industry_securities_report_doc_list_hash(day_list, edicode_values, data_frame)
            write_edinet_numbers_to_txt(industry_securities_report_doc_list_hash, edinet_numbers_dir)
            

        industry_securities_report_doc_list_hash = read_txt_to_industry_securities_report_doc_list_hash(edinet_numbers_dir, data_frame)
        print("start fetching")
        sample_data_dir_with_df_name = call_sample_data_dir(data_frame_name)
        download_xbrl_in_zip(industry_securities_report_doc_list_hash,  sample_data_dir_with_df_name)
        print("download finish")
    except Exception as e:
        with open("error.txt", mode="a") as f:
            f.write(f"{datetime.datetime.now()}: {str(e)} \n") 
        raise e

In [75]:
import pickle

def make_day_list(start_date, end_date):
    
    #取得する書類の日付を指定
    print("start_date：", start_date)
    print("end_day：", end_date)

    period = end_date - start_date
    period = int(period.days)
    day_list = []
    for d in range(period):
        day = start_date + datetime.timedelta(days=d)
        day_list.append(day)

    day_list.append(end_date)
    print("len(day_list):", len(day_list))

    return day_list


def make_edicode_values(data_frame : pd.DataFrame, edinet_col="[EDINETコード]"):
    return list(data_frame[edinet_col].values)


class EdinetDTO:
    
    def __init__(self, edi_code, industry_name, doc_id):
        self.edi_code = edi_code 
        self.industry_name = industry_name
        self.doc_id = doc_id 
    
    def __str__(self):
        return f"edi_code: {self.edi_code},   industry_name: {self.industry_name}, doc_id: {self.doc_id}"


def make_industry_securities_report_doc_list_hash(day_list :list,  edicode_values :list, data_frame: pd.DataFrame,  industry_col="[業種（東証）]", edi_col="[EDINETコード]"):
   

    #コロナ前, コロナ禍, コロナ後全てあるかカウントを計測
    edi_code_count_hash = {}
    edinet_dto_list = make_dto_list(day_list, edicode_values, data_frame, industry_col, edi_col)
    for edinet_dto in edinet_dto_list :
        edi_code  = edinet_dto.edi_code
        if edi_code  not in  edi_code_count_hash :
            edi_code_count_hash[edi_code] = 1
        else :
            edi_code_count_hash[edi_code] += 1
    
    
    industry_securities_report_doc_list_hash = {}
    #edi_codeをもとに、全ての期間を保有する企業のみを抽出
    for  edinet_dto in edinet_dto_list :
        edi_code  = edinet_dto.edi_code
        #コロナ前, コロナ禍, コロナ後全てあるか
        if  edi_code_count_hash[edi_code] <= 3:
            
            industry_name  = edinet_dto.industry_name
            doc_id = edinet_dto.doc_id
            if industry_name not in  industry_securities_report_doc_list_hash :
                industry_securities_report_doc_list_hash[industry_name] = [doc_id]
            else:
                industry_securities_report_doc_list_hash[industry_name].append(doc_id)
              
    print(f"industry_securities_report_doc_list_hash: {industry_securities_report_doc_list_hash}")
    return industry_securities_report_doc_list_hash


def make_dto_list(day_list, edicode_values, data_frame, industry_col,edi_col):
    edinet_dto_list = []
    for index, day in enumerate(day_list):
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        time.sleep(1)
        res = req.get(url, params=params)
        json_data = res.json()
        print(f"{index}/{len(day_list)}")
        
        for num in range(len(json_data["results"])):
            ordinance_code = json_data["results"][num]["ordinanceCode"]
            form_code = json_data["results"][num]["formCode"]

            # 取得したfileが有報かどうか
            if ordinance_code == "010" and form_code == "030000":
                edi_code = json_data["results"][num]["edinetCode"] 
                # 取得したfileが3月期決算企業かどうか
                if edi_code in edicode_values :
#                     industry_name = data_frame[data_frame[edi_col] == edi_code][industry_col].values[0]
                    doc_id = json_data["results"][num]["docID"]
                    edinet_dto = EdinetDTO(edi_code, "サービス業", doc_id)
                    print(edinet_dto)
                    edinet_dto_list.append(edinet_dto)
    
    return  edinet_dto_list


def call_edinet_numbers_dir(data_frame_name: str):
    #一時的に一覧取得するためのIDを保存
    
    edinet_numbers_dir = os.getcwd()+  "/EdinetNumbers"
    if not os.path.exists(edinet_numbers_dir) :
        os.mkdir(edinet_numbers_dir)

    edinet_numbers_dir = os.getcwd()+  "/EdinetNumbers"+ f"/{data_frame_name}"
    if not os.path.exists(edinet_numbers_dir) :
        os.mkdir(edinet_numbers_dir)
        
    return edinet_numbers_dir






def write_edinet_numbers_to_txt(industry_securities_report_doc_list_hash,  edinet_numbers_dir):
    values = list(industry_securities_report_doc_list_hash.values())[0]
    with open(f"{edinet_numbers_dir}/サービス業_edinet_numbers", "wb") as fp:
        pickle.dump(values, fp)

def read_txt_to_industry_securities_report_doc_list_hash(edinet_numbers_dir, data_frame, industry_col="[業種（東証）]"):
    
    industry_securities_report_doc_list_hash = {}
    
    #industry_list = list(set(data_frame[industry_col].values))
    industry_list = ["サービス業"]
    for industry_name in industry_list:        
        with open(f"{edinet_numbers_dir}/サービス業_edinet_numbers", "rb") as fp: 
            li = pickle.load(fp)
    
        industry_securities_report_doc_list_hash[industry_name] = li
    
    return industry_securities_report_doc_list_hash


def call_sample_data_dir(data_frame_name: str):
    
    #Sampleフォルダの作成
    sample_data_dir = os.getcwd()+  "/SampleData"
    if not os.path.exists(sample_data_dir) :
        os.mkdir(sample_data_dir)
        print(f"created: {sample_data_dir}")
    
    sample_data_dir_with_df_name =  sample_data_dir + f"/{data_frame_name}"
    if not os.path.exists(sample_data_dir_with_df_name) :
        os.mkdir(sample_data_dir_with_df_name)
    return sample_data_dir_with_df_name


def download_xbrl_in_zip(industry_securities_report_doc_list_hash,  sample_data_dir_with_df_name):
    
    sum_files_len = sum(map(len, industry_securities_report_doc_list_hash.values()))
    v_len = 0
    
    for k, v in industry_securities_report_doc_list_hash.items():
        
        filepath_edinet = call_industry_dir(sample_data_dir_with_df_name, k)
        v_len += len(v)
        for index, doc_id in enumerate(v):
            print(f"{doc_id} :{ index + 1} /{len(v)}  | {v_len} /{sum_files_len}")
            url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + doc_id
            params = {"type": 1}
            filename = filepath_edinet + "/"  + doc_id + ".zip"
            if os.path.exists(filename) :
                continue
            else:
                #負荷を与えないために2秒間待つ
                time.sleep(2)
                res = req.get(url, params=params, stream=True)

                if res.status_code == 200:
                    with open(filename, 'wb') as file:
                        for chunk in res.iter_content(chunk_size=1024):
                            file.write(chunk)
                            

def call_industry_dir( sample_data_dir_with_df_name, industry_name):
    industry_dir = sample_data_dir_with_df_name +  f"/{industry_name}"
    if not os.path.exists(industry_dir) :
        os.mkdir(industry_dir)
        print(f"created: {industry_dir}")
    return industry_dir

In [76]:
#今回は新型コロナウイルス感染症を意識した期間を設定
  # コロナ前(2019年3月期決算: 2018年4月1日~2019年3月31日)
  # コロナ過渡期(2019年4月~2020年3月)
  # コロナ後(2020年4月~2021年3月)
start_date = datetime.date(2018, 4, 1)
end_date = datetime.date(2021, 3,31)
day_list = make_day_list(start_date, end_date)

start_date： 2018-04-01
end_day： 2021-03-31
len(day_list): 1096


In [77]:
#今回は2021年度時点での3月期決算企業のEDINETコードが記載されたcsvファイルを使用(hirenketsu or renketsuを指定)
# data_frame_name = "ave_top_20"
# data_frame_name = "ave_worst_20"
# data_frame_name = "2021_top_20"
data_frame_name = "2021_worst_20"

data_frame = pd.read_csv(f"./EdinetIdxFiles/{data_frame_name}.csv", skiprows=0)

In [84]:
call_edinet_api(data_frame, data_frame_name, start_date, end_date)

start fetching
created: /home/jovyan/work/1CallingEdinetApi/SampleData/2021_worst_20/サービス業
S100D8C6 :1 /111  | 111 /111
S100D96E :2 /111  | 111 /111
S100D8EW :3 /111  | 111 /111
S100D9AN :4 /111  | 111 /111
S100DA5I :5 /111  | 111 /111
S100DBF5 :6 /111  | 111 /111
S100DC27 :7 /111  | 111 /111
S100DEFP :8 /111  | 111 /111
S100DEKM :9 /111  | 111 /111
S100DE67 :10 /111  | 111 /111
S100DFPP :11 /111  | 111 /111
S100DHEA :12 /111  | 111 /111
S100DH1D :13 /111  | 111 /111
S100DHNZ :14 /111  | 111 /111
S100DHFZ :15 /111  | 111 /111
S100DHUH :16 /111  | 111 /111
S100DBAI :17 /111  | 111 /111
S100DGZY :18 /111  | 111 /111
S100DFEM :19 /111  | 111 /111
S100D6H5 :20 /111  | 111 /111
S100DIBN :21 /111  | 111 /111
S100DHWN :22 /111  | 111 /111
S100DDTD :23 /111  | 111 /111
S100DIUM :24 /111  | 111 /111
S100DI22 :25 /111  | 111 /111
S100DHYH :26 /111  | 111 /111
S100DH01 :27 /111  | 111 /111
S100DIN9 :28 /111  | 111 /111
S100DJ1A :29 /111  | 111 /111
S100DJ27 :30 /111  | 111 /111
S100D73F :31 /111 

In [60]:
data_frame_name

'2021_top_20'

In [82]:
edinet_numbers_dir = call_edinet_numbers_dir(data_frame_name)
edinet_numbers_dir 

'/home/jovyan/work/1CallingEdinetApi/EdinetNumbers/2021_worst_20'

In [83]:
a = ['S100D8C6', 'S100D96E', 'S100D8EW', 'S100D9AN', 'S100DA5I', 'S100DBF5', 'S100DC27', 'S100DEFP', 'S100DEKM', 'S100DE67', 'S100DFPP', 'S100DHEA', 'S100DH1D', 'S100DHNZ', 'S100DHFZ', 'S100DHUH', 'S100DBAI', 'S100DGZY', 'S100DFEM', 'S100D6H5', 'S100DIBN', 'S100DHWN', 'S100DDTD', 'S100DIUM', 'S100DI22', 'S100DHYH', 'S100DH01', 'S100DIN9', 'S100DJ1A', 'S100DJ27', 'S100D73F', 'S100DHQO', 'S100DJFL', 'S100DJ8I', 'S100DIN5', 'S100DJJQ', 'S100DQ13', 'S100G07V', 'S100G34Q', 'S100G2QI', 'S100G3LB', 'S100G1EU', 'S100G51C', 'S100G5FT', 'S100G61Y', 'S100G7UG', 'S100G40R', 'S100G9HI', 'S100G8HM', 'S100G8DY', 'S100G7VF', 'S100G5P7', 'S100G2ZS', 'S100GBGK', 'S100GBM9', 'S100G4WX', 'S100G51U', 'S100GB0O', 'S100G83B', 'S100GCVG', 'S100G9BD', 'S100GCXU', 'S100GCWW', 'S100GCQV', 'S100GCOY', 'S100GCMA', 'S100GBUS', 'S100GDGC', 'S100GDHB', 'S100GC9R', 'S100GDDX', 'S100GCH9', 'S100GCO4', 'S100GDWJ', 'S100ITLI', 'S100IV45', 'S100IVUV', 'S100IWL2', 'S100IYEC', 'S100IYA9', 'S100IWQ4', 'S100IZVE', 'S100IZGZ', 'S100IX1Z', 'S100IZQ7', 'S100J1D1', 'S100J1I5', 'S100IUPZ', 'S100J17N', 'S100IWME', 'S100J2GO', 'S100J2LX', 'S100J2J9', 'S100J0WA', 'S100IV2P', 'S100IXEE', 'S100J31L', 'S100J24Q', 'S100J3JN', 'S100J4TF', 'S100J4PI', 'S100J4WM', 'S100J4XQ', 'S100J4XU', 'S100J4CQ', 'S100J5J9', 'S100JAT9', 'S100JBHU', 'S100JE4A', 'S100JJP4', 'S100JN7W']
edinet_numbers_dir = call_edinet_numbers_dir(data_frame_name)

with open(f"{edinet_numbers_dir}/サービス業_edinet_numbers", "wb") as fp:
        pickle.dump(a, fp)

In [80]:
with open(f"{edinet_numbers_dir}/サービス業_edinet_numbers", "rb") as fp: 
            li = pickle.load(fp)
li

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/1CallingEdinetApi/EdinetNumbers/2021_worst_20/サービス業_edinet_numbers'